## Automatización de Procesos Usando Selenium en Python

### Autor: Jorge Sanisaca

### ¿Qué es Selenium?
Selenium es una suit de herramientas para automatizar lo que sucede en un borwser, es un software libre

### Prerrequisitos
- python 3.7 o supeior
- web driver de selenium

### Instalaciones necesarias
- ```pip install selenium```
- ```pip install openpylx```

### Proceso

Una vez isntaladas las librerias necesarias se procede a importarlas

In [2]:
# Importar librerias
import time
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import os
import openpyxl
from openpyxl import Workbook
import requests
import json

#### Automatizacion de insercion de datos

El siguiente código se usa para registrar un tipo de geografia y agregar una ruta (compra de boleto) en el sistema, Para ello procederemos a importar el ```webdriver``` de selenium que es el que controlará nuestro navegador e irá buscando el compo que deseemos y lo llenará automaticamente, una vez que los campos estén completos estos se registrarán automaticamente en la base de datos del sistema.

In [3]:
driver = webdriver.Chrome('chromedriver.exe')

# Aceder a la pagina
driver.get('http://localhost:4200')
time.sleep(4)

# Agregar Tipo Geografia
link_tipo_geografia = driver.find_element_by_xpath("//*[@id='navbarSupportedContent']/ul/li[2]/a")
link_tipo_geografia.click()
time.sleep(4)
codigo_tipo_geografia = driver.find_element_by_xpath("/html/body/app-root/app-tipo-geografia/div/form/div[1]/div[1]/input")
codigo_tipo_geografia.send_keys('05')
time.sleep(3)
descripcion_tipo_geografia = driver.find_element_by_xpath("/html/body/app-root/app-tipo-geografia/div/form/div[1]/div[2]/input")
descripcion_tipo_geografia.send_keys('Barrio')
time.sleep(3)
guardar_tipo_geografia =driver.find_element_by_xpath("/html/body/app-root/app-tipo-geografia/div/form/div[2]/div[2]/button")
guardar_tipo_geografia.click()
time.sleep(4)

# Agregar Geografia
link_ruta = driver.find_element_by_xpath("//*[@id='navbarSupportedContent']/ul/li[4]/a")
link_ruta.click()
time.sleep(4)
codigo_ruta = driver.find_element_by_xpath("/html/body/app-root/app-ruta/div/form/div/div[1]/div[1]/input")
codigo_ruta.send_keys('01')
time.sleep(3)
descripcion_ruta = driver.find_element_by_xpath("/html/body/app-root/app-ruta/div/form/div/div[1]/div[2]/input")
descripcion_ruta.send_keys('Regreso')
time.sleep(3)
origen_dropDown = Select(driver.find_element_by_xpath("/html/body/app-root/app-ruta/div/form/div/div[2]/div[1]/select"))
origen_dropDown.select_by_index(3)
time.sleep(3)
destino_dropDown = Select(driver.find_element_by_xpath("/html/body/app-root/app-ruta/div/form/div/div[2]/div[2]/select"))
destino_dropDown.select_by_index(4)
time.sleep(3)
valor_ruta = driver.find_element_by_xpath("/html/body/app-root/app-ruta/div/form/div/div[2]/div[3]/input")
valor_ruta.send_keys('8.25')
time.sleep(3)
guardar_ruta = driver.find_element_by_xpath("/html/body/app-root/app-ruta/div/form/div/div[3]/div[2]/button")
guardar_ruta.click()
time.sleep(4)


#### Guardar datos de una pagina web en un archivo
A continuación, trabajando con el mismo sitio web, procedemos a la manipulacion de archivos es decir, extraeremos la información de las rutas y la guardaremos en un archivo de Excel.

In [4]:
rows_count = len(driver.find_elements_by_xpath("/html/body/app-root/app-ruta/div/table/tbody/tr"))
cols_count = len(driver.find_elements_by_xpath("/html/body/app-root/app-ruta/div/table/tbody/tr[1]/td"))

print("Number of Rows: ",rows_count)
print("Number of Cols: ",cols_count)

first_part = "/html/body/app-root/app-ruta/div/table/tbody/tr["
second_part = "]/td["
third_part = "]"

for n in range(1,rows_count+1):
    for m in range(1,cols_count):
        final_path = first_part+str(n)+second_part+str(m)+third_part
        table_data = driver.find_element_by_xpath(final_path).text
        fname = "RegistroRutas.xlsx"
        if(os.path.exists(fname)):
            workbook = openpyxl.load_workbook(fname)
            worksheet = workbook["Sheet"]
        else:
            workbook = Workbook()
            worksheet = workbook.active
        worksheet.cell(row=n, column=m).value = table_data
        workbook.save(fname)

Number of Rows:  5
Number of Cols:  6


#### Uso del servicio de Open Weather Map
Finalmente consumiremos un servicio de Open Weather Mar para extraer la Temperatura, Presion Atmosférica, Humedad de un determinado país o ciudad.

In [5]:
base_url = "http://api.openweathermap.org/data/2.5/weather"
api_key="42a5f6dfd359089d448b0dabe07a25b6"

# Ingresamos el nombre de la ciudad
city_name = input("Ingrese el Nombre de la Ciudad: ")

# Generamos la url completa
complete_url = base_url + "?appid=" + api_key + "&q=" + city_name

driver.get(complete_url)
time.sleep(4)

#get the ui element
ui_element=driver.find_element_by_tag_name("pre")
#get the weather data
weather_data=ui_element.text
# print(weather_data)

# Transformamos los datos obtenidos a JSON
json_data=json.loads(weather_data)

if json_data["cod"] != "404":
    y = json_data["main"] 
    
    current_temperature = y["temp"] 
    current_pressure = y["pressure"]
    current_humidiy = y["humidity"]
    
    z = json_data["sys"]
    
    country = z["country"]
    
    print("\nPais = "+str(country))
    print("Temperatura (Kelvil) = " + str(current_temperature)) 
    print("Presion Atmosferica (hPa) = " +str(current_pressure))
    print("Humedad (%) = "+str(current_humidiy))
else: 
    print("Ciudad No Encontrada") 

Ingrese el Nombre de la Ciudad: Cuenca

Pais = EC
Temperatura (Kelvil) = 283.15
Presion Atmosferica (hPa) = 1025
Humedad (%) = 87
